In [2]:
using Revise

In [3]:
using Flux
using JLD2
using FileIO
using MLDataPattern
using CoordinateTransformations
using Plots; gr()

Plots.GRBackend()

In [48]:
using ProgressMeter

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/ProgressMeter.ji for module ProgressMeter.


In [4]:
import FluxExtensions
import LearningMPC

In [5]:
samples = load("2017-12-20-hopper-grid/grid_search.jld2")["samples"];

In [6]:
features(sample::LearningMPC.Sample) = (sample.state, sample.mip.objective_bound, sample.mip.objective_value)

features (generic function with 1 method)

In [7]:
data = features.(samples)

145656-element Array{Tuple{Array{Float64,1},Float64,Float64},1}:
 ([0.25, 0.25, -5.0, -5.0], 210.503, 232.332)
 ([0.45, 0.25, -5.0, -5.0], 226.435, 246.457)
 ([0.65, 0.25, -5.0, -5.0], 263.729, 277.768)
 ([0.85, 0.25, -5.0, -5.0], 298.823, 298.823)
 ([1.05, 0.25, -5.0, -5.0], 251.796, 262.749)
 ([1.25, 0.25, -5.0, -5.0], 296.06, 296.06)  
 ([1.45, 0.25, -5.0, -5.0], 295.977, 295.977)
 ([1.65, 0.25, -5.0, -5.0], 315.658, 316.245)
 ([1.85, 0.25, -5.0, -5.0], 333.866, 333.866)
 ([2.05, 0.25, -5.0, -5.0], 431.564, 431.564)
 ([2.25, 0.25, -5.0, -5.0], 426.183, 426.183)
 ([2.45, 0.25, -5.0, -5.0], 645.097, 645.097)
 ([2.65, 0.25, -5.0, -5.0], 500.212, 500.242)
 ⋮                                           
 ([0.65, 0.85, 5.0, 5.0], 29.9035, 31.0021)  
 ([0.85, 0.85, 5.0, 5.0], 36.1409, 36.3911)  
 ([1.05, 0.85, 5.0, 5.0], 13.1182, 13.2453)  
 ([1.25, 0.85, 5.0, 5.0], 13.1706, 13.2987)  
 ([1.45, 0.85, 5.0, 5.0], 36.2983, 36.4273)  
 ([1.65, 0.85, 5.0, 5.0], 82.5012, 82.6312)  
 ([1.85, 0.85, 

In [8]:
train_data, test_data = splitobs(shuffleobs(data), at=0.85)

(Tuple{Array{Float64,1},Float64,Float64}[([1.45, 0.45, -2.4, 2.4], 241.228, 260.02), ([1.05, 0.85, -0.2, 3.4], 228.441, 228.441), ([2.65, 0.65, -3.0, -2.0], 547.303, 547.343), ([0.45, 0.85, 3.4, -3.2], 54.3152, 57.8058), ([2.45, 0.85, -2.6, 3.6], 442.608, 442.813), ([0.25, 0.65, -4.4, 0.8], 258.173, 283.18), ([1.65, 0.85, -2.6, -5.0], 255.95, 255.96), ([0.85, 0.85, -3.2, 2.6], 186.381, 186.381), ([2.85, 0.45, 1.2, 2.0], 44.7475, 44.857), ([0.85, 0.45, 1.6, 0.0], 279.266, 296.382)  …  ([0.85, 0.85, 1.2, 1.6], 235.669, 240.508), ([2.05, 0.65, 2.2, -4.6], 7.23971, 7.32992), ([2.25, 0.45, 0.4, -1.2], 152.825, 152.909), ([1.65, 0.65, -2.8, 1.6], 254.917, 254.949), ([2.45, 0.85, -3.6, -0.8], 367.5, 367.5), ([2.45, 0.65, 1.6, -1.6], 20.1627, 20.2578), ([2.85, 0.45, -0.6, -4.8], 94.3499, 94.3899), ([1.25, 0.45, 4.2, 1.2], 7.91228, 8.01748), ([1.45, 0.45, 4.4, 0.4], 22.7554, 22.8575), ([2.65, 0.25, -3.6, -4.6], 421.931, 421.931)], Tuple{Array{Float64,1},Float64,Float64}[([1.25, 0.45, 0.8, -2.2]

In [9]:
train_data

123808-element SubArray{Tuple{Array{Float64,1},Float64,Float64},1,Array{Tuple{Array{Float64,1},Float64,Float64},1},Tuple{Array{Int64,1}},false}:
 ([1.45, 0.45, -2.4, 2.4], 241.228, 260.02)  
 ([1.05, 0.85, -0.2, 3.4], 228.441, 228.441) 
 ([2.65, 0.65, -3.0, -2.0], 547.303, 547.343)
 ([0.45, 0.85, 3.4, -3.2], 54.3152, 57.8058) 
 ([2.45, 0.85, -2.6, 3.6], 442.608, 442.813) 
 ([0.25, 0.65, -4.4, 0.8], 258.173, 283.18)  
 ([1.65, 0.85, -2.6, -5.0], 255.95, 255.96)  
 ([0.85, 0.85, -3.2, 2.6], 186.381, 186.381) 
 ([2.85, 0.45, 1.2, 2.0], 44.7475, 44.857)   
 ([0.85, 0.45, 1.6, 0.0], 279.266, 296.382)  
 ([2.45, 0.85, -1.2, -1.0], 356.934, 356.984)
 ([0.45, 0.45, 0.8, -5.0], 103.174, 108.784) 
 ([1.45, 0.45, -2.2, 0.6], 261.089, 261.089) 
 ⋮                                           
 ([0.65, 0.45, 3.0, -4.8], 183.88, 183.962)  
 ([0.85, 0.85, 0.8, -4.2], 245.874, 245.874) 
 ([0.85, 0.85, 1.2, 1.6], 235.669, 240.508)  
 ([2.05, 0.65, 2.2, -4.6], 7.23971, 7.32992) 
 ([2.25, 0.45, 0.4, -1.2], 

In [206]:
function setup_model()
    model = Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 32, elu),
        Dense(32, 32, elu),
        Dense(32, 1, elu),
        AffineMap(fill(750, 1, 1), [750])
        )

    sample_loss = (x, lb, ub) -> begin
        y = model(x)
        sum(ifelse.(y .< lb, lb .- y, ifelse.(y .> ub, y .- ub, 0 .* y)))
    end
    batch_loss = (samples) -> sum((sample -> sample_loss(sample...)).(samples)) / length(samples)
    model, batch_loss
end

model, loss = setup_model()

(Chain(LinearMap(UniformScaling{Float64}
0.2*I), Dense(4, 32, NNlib.elu), Dense(32, 32, NNlib.elu), Dense(32, 1, NNlib.elu), AffineMap([750], [750])), #195)

In [222]:
model(train_data[1][1])

Tracked 1-element Array{Float64,1}:
 261.781

In [223]:
train_data[1]

([1.45, 0.45, -2.4, 2.4], 241.2277002239117, 260.020092681482)

In [208]:
opt = Flux.ADADelta(params(model))

(::#71) (generic function with 1 method)

In [209]:
@showprogress for i in 1:200
    batches = eachbatch(shuffleobs(train_data), size=50)
    for batch in batches
        l = loss(batch)
        isinf(Flux.Tracker.value(l)) && error("Loss is Inf")
        isnan(Flux.Tracker.value(l)) && error("Loss is NaN")
        Flux.back!(l)
        opt()
    end
    @show loss(train_data)
end

loss(train_data) = param(78.8085)


Progress:   0%|                                         |  ETA: 3:50:08

loss(train_data) = param(75.4834)


Progress:   2%|█                                        |  ETA: 3:41:41

loss(train_data) = param(73.6842)
loss(train_data) = param(71.2093)


...

Progress:  98%|████████████████████████████████████████ |  ETA: 0:03:27

loss(train_data) = param(14.1833)
loss(train_data) = param(16.2201)


Progress: 100%|█████████████████████████████████████████|  ETA: 0:01:09

loss(train_data) = param(13.9542)
loss(train_data) = param(13.6017)


Progress: 100%|█████████████████████████████████████████| Time: 3:49:44


In [210]:
loss(train_data)

Tracked 0-dimensional Array{Float64,0}:
13.6017

In [211]:
loss(test_data)

Tracked 0-dimensional Array{Float64,0}:
13.5435

In [218]:
jldopen("2018-01-19-flux-value-function-data.jld2", "w") do file
    file["model"] = model
    file["params"] = params(model)
end

6-element Array{Any,1}:
 param([-2.10231 -0.416695 -0.580282 0.0422408; -0.754705 0.662337 -0.143734 0.0488835; … ; -1.10827 0.668861 0.0487905 -0.0252331; -1.30875 -0.180811 -2.26249 0.205947])                                                               
 param([0.844484, 0.178616, 0.403009, -0.208104, 0.49962, 0.44575, 0.266903, -0.51562, 1.32075, 0.369469  …  -1.04144, 1.22526, -0.0552373, -0.0680415, 0.171739, -0.303844, -0.218562, -0.339674, 0.672745, 1.77431])                   
 param([-0.982463 -0.204259 … -0.865279 0.10621; -0.771774 0.0760247 … -0.656337 -0.153835; … ; -0.759916 -0.563214 … 0.197194 -0.426359; 0.177251 0.0270011 … -0.293825 0.0485714])                                                     
 param([0.0213042, -0.19165, -0.569993, -0.0782762, -0.0249571, -0.0459417, -0.113423, 0.0118172, 0.24209, -0.0710401  …  0.0657258, -0.0926567, 0.135144, -0.344255, -0.101306, -0.502183, -0.825598, -0.419729, -0.232764, 0.00338587])
 param([0.747778 0.224254 … 0.586545 -0.

In [212]:
import LCPSim
import Hoppers
using RigidBodyDynamics
using Gurobi
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

true

In [213]:
robot = Hoppers.Hopper()
xstar = Hoppers.nominal_state(robot)
ustar = zeros(num_velocities(xstar))
basevis = Visualizer()[:hopper]
setgeometry!(basevis, robot)
settransform!(basevis[:robot], xstar)

Q, R = Hoppers.default_costs(robot)
foot = findbody(robot.mechanism, "foot")
Δt = 0.05
Jc = LCPSim.ContactLQR.contact_jacobian(xstar, [Point3D(default_frame(foot), 0., 0., 0.)])
A, B, c = LCPSim.ContactLQR.contact_linearize(xstar, ustar, Jc)

([0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], [0.0 0.0; 0.0 0.0; 0.0 1.0; 0.0 1.0], [0.0, 0.0, -9.81, -9.81])

In [214]:
tangent_net = FluxExtensions.TangentPropagator(model)
net_value_controller = state -> begin
    x = state_vector(state)
    value, jac = tangent_net(x)
    u = vec(-inv(R) * B' * Flux.Tracker.value(jac)')
end

(::#201) (generic function with 1 method)

In [220]:
x_init = MechanismState{Float64}(robot.mechanism)
set_configuration!(x_init, [1.0, 1.0])
set_velocity!(x_init, [0., 0.])
LearningMPC.randomize!(x_init, x_init, 0.5, 1.0)
results = LCPSim.simulate(x_init, net_value_controller,
    robot.environment,
    Δt,
    100,
    GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [221]:
LearningMPC.playback(basevis[:robot], results, Δt)

In [164]:
x = [1., 1., 1., 1]
tangent_net(x)

(param([213.646]), param([160.984 -46.8352 9.64681 -0.567878]))

In [205]:
net_value_controller(x_init)

2-element Array{Float64,1}:
     0.0 
 -5203.69